### Load the libraries/modules

In [224]:
import os, sys, copy, time
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import statsmodels.api as sm
import functions as func

### Load the necessary data

In [225]:
# Load all the necessary data
nifty50_bc = pd.read_csv('./../data/nifty50_before_covid.csv')
nifty50_ac = pd.read_csv('./../data/nifty50_after_covid.csv')
firms_bc = pd.read_csv('./../data/map_data_before_covid.csv')
firms_ac = pd.read_csv('./../data/map_data_after_covid.csv')
banks_vulnerability = pd.read_csv('./../data/banks_vulnerability.csv', index_col=0)
banks_vulnerability.columns = ['symbol', 'banker_name', 'type', 'beta', 'mes', 'vulnerable']
banks_vulnerability['banker_name'] = banks_vulnerability['banker_name'].apply(lambda x: x.upper())
banks_firms_connection = pd.read_csv('./../data/bank_firm_connection_filter.csv')
banks_firms_connection.drop(['bnkhist_date'], axis = 1, inplace = True)


### Date-time column to datetime object

In [226]:
nifty50_bc['Date'] = pd.to_datetime(nifty50_bc['Date'])
nifty50_ac['Date'] = pd.to_datetime(nifty50_ac['Date'])
firms_bc['co_stkdate'] = pd.to_datetime(firms_bc['co_stkdate'])
firms_bc['co_stkdate'] = pd.to_datetime(firms_bc['co_stkdate'])

### Demo

In [227]:
# Demo for company: 20 MICRONS LTD.
firm_cp = firms_bc.loc[firms_bc['company_name'] == '20 MICRONS LTD.']['bse_closing_price']
firm_cp.reset_index(drop = True, inplace = True)
reg_data = func.making_data_reg(firm_cp, nifty50_bc['Close'])
reg_data.dropna(inplace=True)
reg_data

,y,x-2,x-1,x0,x+1,x+2
3,-0.010283,-0.021382,-0.011350,0.002517,0.000542,0.003767
4,-0.022078,-0.000234,-0.021382,-0.011350,0.002517,0.000542
5,-0.039841,-0.004933,-0.000234,-0.021382,-0.011350,0.002517
6,0.006916,0.007305,-0.004933,-0.000234,-0.021382,-0.011350
7,-0.013736,-0.002625,0.007305,-0.004933,-0.000234,-0.021382
...,...,...,...,...,...,...
116,0.015504,-0.003931,-0.000737,0.000987,0.003113,0.004657
117,-0.016794,-0.007205,-0.003931,-0.000737,0.000987,0.003113
118,-0.007764,0.009834,-0.007205,-0.003931,-0.000737,0.000987
119,0.004695,0.000821,0.009834,-0.007205,-0.003931,-0.000737


In [228]:
func.crash_risk(reg_data.drop(['y'], axis=1).values, reg_data['y'].values)

(0, 0.00012811874008187898, 0.19431897154488348)

### Calculating crash risk

Running over all firms:

Limitation: All firms do not have much data.
- Run for those firms which have exact 120 days data available.
- NSE/BSE (Which have more data)

In [229]:
# Printing the shape of the data
print(f'Shape of the mapping data is (Before corona): {firms_bc.shape}')
print(f'Shape of the mapping data is (After corona): {firms_ac.shape}')

Shape of the mapping data is (Before corona): (241296, 6)
Shape of the mapping data is (After corona): (240115, 6)


In [230]:
# Filtered out by exchange center which have atleast 100 days data
df_nse = firms_bc.dropna(thresh=2, axis=0).groupby(by = ['company_name'])['nse_closing_price'].count().reset_index()
df_nse = df_nse[df_nse.nse_closing_price > 100]
df_bse = firms_bc.dropna(thresh=2, axis=0).groupby(by = ['company_name'])['bse_closing_price'].count().reset_index()
df_bse = df_bse[df_bse.bse_closing_price > 100]

In [231]:
# Separate out by exchange center
company_name_nse = df_nse['company_name'].values
company_name_bse = []
for company in df_bse['company_name'].values:
    if company not in company_name_nse:
        company_name_bse.append(company)

In [232]:
result_crash_risk_bc = []
firms_singular_bc = []
# NSE: Before COVID
for firm in tqdm(company_name_nse):
    try:
        firm_cp = firms_bc.loc[firms_bc['company_name'] == firm]['nse_closing_price']
        firm_cp.reset_index(drop = True, inplace = True)
        reg_data = func.making_data_reg(firm_cp, nifty50_bc['Close'])
        reg_data.dropna(inplace=True)
        result = func.crash_risk(reg_data.drop(['y'], axis=1).values, reg_data['y'].values)
        result_crash_risk_bc.append(('NSE', firm, 0, result[0], result[1], result[2]))
    except:
        firms_singular_bc.append(firm)
# BSE: Before COVID
for firm in tqdm(company_name_bse):
    try:
        firm_cp = firms_bc.loc[firms_bc['company_name'] == firm]['bse_closing_price']
        firm_cp.reset_index(drop = True, inplace = True)
        reg_data = func.making_data_reg(firm_cp, nifty50_bc['Close'])
        reg_data.dropna(inplace=True)
        result = func.crash_risk(reg_data.drop(['y'], axis=1).values, reg_data['y'].values)
        result_crash_risk_bc.append(('BSE', firm, 0, result[0], result[1], result[2]))
    except:
        firms_singular_bc.append(firm)
result_crash_risk_bc = pd.DataFrame(result_crash_risk_bc, columns=['exchange','company_name','time','binary','nskew','duvol'])

  0%|          | 0/1082 [00:00<?, ?it/s]

  0%|          | 0/526 [00:00<?, ?it/s]

d:\Term Project\notebook\functions.py:59: RuntimeWarning: invalid value encountered in double_scalars
  return -numerator/denominator
d:\Term Project\notebook\functions.py:76: RuntimeWarning: invalid value encountered in double_scalars
  return np.log(numerator/denominator)


In [233]:
result_crash_risk_ac = []
firms_singular_ac = []
# NSE: During COVID
for firm in tqdm(company_name_nse):
    try:
        firm_cp = firms_ac.loc[firms_ac['company_name'] == firm]['nse_closing_price']
        firm_cp.reset_index(drop = True, inplace = True)
        reg_data = func.making_data_reg(firm_cp, nifty50_bc['Close'])
        reg_data.dropna(inplace=True)
        result = func.crash_risk(reg_data.drop(['y'], axis=1).values, reg_data['y'].values)
        result_crash_risk_ac.append(('NSE', firm, 1, result[0], result[1], result[2]))
    except:
        firms_singular_ac.append(firm)
# BSE: During COVID
for firm in tqdm(company_name_bse):
    try:
        firm_cp = firms_ac.loc[firms_ac['company_name'] == firm]['bse_closing_price']
        firm_cp.reset_index(drop = True, inplace = True)
        reg_data = func.making_data_reg(firm_cp, nifty50_bc['Close'])
        reg_data.dropna(inplace=True)
        result = func.crash_risk(reg_data.drop(['y'], axis=1).values, reg_data['y'].values)
        result_crash_risk_ac.append(('BSE', firm, 1, result[0], result[1], result[2]))
    except:
        firms_singular_ac.append(firm)
result_crash_risk_ac = pd.DataFrame(result_crash_risk_ac, columns=['exchange','company_name','time','binary','nskew','duvol'])

  0%|          | 0/1082 [00:00<?, ?it/s]

  0%|          | 0/526 [00:00<?, ?it/s]

In [234]:
final_result = pd.concat([result_crash_risk_bc, result_crash_risk_ac], axis=0)
final_result.sort_values(by = ['company_name'], inplace = True)
final_result = final_result[~final_result['company_name'].isin(firms_singular_bc)]
final_result = final_result[~final_result['company_name'].isin(firms_singular_ac)]

In [235]:
final_result = pd.merge(final_result, banks_firms_connection, on = 'company_name')
final_result = pd.merge(final_result, banks_vulnerability, on = 'banker_name')
final_result = final_result.loc[:,['symbol','banker_name','co_code','company_name','exchange','beta','mes','vulnerable','type','time','binary','nskew','duvol']]
final_result

,symbol,banker_name,co_code,company_name,exchange,beta,mes,vulnerable,type,time,binary,nskew,duvol
0,SBIN.NS,STATE BANK OF INDIA,11,20 MICRONS LTD.,NSE,0.247401,0.001617,True,Public,0,0,-0.031853,-0.306464
1,SBIN.NS,STATE BANK OF INDIA,11,20 MICRONS LTD.,NSE,0.247401,0.001617,True,Public,1,1,0.009624,-0.154393
2,SBIN.NS,STATE BANK OF INDIA,389178,A 2 Z INFRA ENGG. LTD.,NSE,0.247401,0.001617,True,Public,1,1,0.001683,0.094062
3,SBIN.NS,STATE BANK OF INDIA,389178,A 2 Z INFRA ENGG. LTD.,NSE,0.247401,0.001617,True,Public,0,0,-0.079382,0.048425
4,SBIN.NS,STATE BANK OF INDIA,289,A B C INDIA LTD.,BSE,0.247401,0.001617,True,Public,0,0,0.009701,-0.045269
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3209,KTKBANK.NS,KARNATAKA BANK LTD.,165297,ORTEL COMMUNICATIONS LTD.,NSE,0.098743,0.001626,True,Private,0,0,0.018751,0.538088
3210,KTKBANK.NS,KARNATAKA BANK LTD.,165319,ORTIN LABORATORIES LTD.,NSE,0.098743,0.001626,True,Private,0,0,-0.092344,-0.332753
3211,KTKBANK.NS,KARNATAKA BANK LTD.,165319,ORTIN LABORATORIES LTD.,NSE,0.098743,0.001626,True,Private,1,0,-0.106369,-0.864495
3212,CUB.NS,CITY UNION BANK LTD.,176994,POLYSPIN EXPORTS LTD.,BSE,0.084847,-0.002707,True,Private,1,0,-0.034252,-0.305465


In [236]:
final_result.to_csv('./../data/final_result.csv', index = False)